In [1]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import dask_cudf
import dask
import cudf
import glob

data_path_in = "/datasets/criteo/raw_csvs/split_train_data"
aggregate_files = True
num_files = None # Set to None to read all files
num_output_files = None
row_group_size = None
column_count = 40

if aggregate_files:
    data_path_out = "/datasets/criteo/raw_csvs/split_train_data_parquet_agg"
    num_output_files = 1400
    row_group_size = 2000000
else:
    data_path_out = "/datasets/criteo/raw_csvs/split_train_data_parquet"

/home/nfs/rzamora/miniconda3/envs/ucx/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/nfs/rzamora/miniconda3/envs/ucx/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


### Create a cluster of GPU workers

In [2]:
cluster = LocalCUDACluster(device_memory_limit="32GB")
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41669 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 1.08 TB


### Read in Criteo Dataset, and Calculate Statistics

In [3]:
files = glob.glob(data_path_in + "/*")
if num_files and num_files < len(files):
    files = files[:num_files]

names = ["Label"] + ["I"+str(i+1) for i in range(13)] + ["C"+str(i+1) for i in range(26)]
dtype = ["int64"] + ["int64" for i in range(13)] + ["str" for i in range(26)]
ddf = dask_cudf.read_csv(files, delimiter="\t", dtype=dtype, names=names)
if num_output_files:
    ddf = ddf.repartition(npartitions=num_output_files)
else:
    row_group_size = None

First, check the first few rows

In [4]:
ddf.head()

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,5,110,null,16,null,1,0,14,7,...,d20856aa,b8170bba,9512c20b,c38e2f28,14f65a5d,25b1b089,d7c1fc0b,7caf609c,30436bfc,ed10571d
1,0,32,3,5,null,1,0,0,61,5,...,d20856aa,a1eb1511,9512c20b,febfd863,a3323ca1,c8e1ee56,1752e9e8,75350c8a,991321ea,b757e957
2,0,null,233,1,146,1,0,0,99,7,...,d20856aa,628f1b8d,9512c20b,c38e2f28,14f65a5d,25b1b089,d7c1fc0b,34a9b905,ff654802,ed10571d
3,0,null,24,null,11,24,null,0,56,3,...,1f7fc70b,a1eb1511,9512c20b,None,None,None,dc209cd3,b8a81fb0,30436bfc,b757e957
4,0,60,223,6,15,5,0,0,1,8,...,d20856aa,d9f758ff,9512c20b,c709ec07,2b07677e,a89a92a5,aa137169,e619743b,cdc3217e,ed10571d


Convert csv files to parquet files

In [5]:
%time ddf.to_parquet(data_path_out, row_group_size=row_group_size)

CPU times: user 7min 17s, sys: 31.2 s, total: 7min 48s
Wall time: 57min 21s
